In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F

pd.set_option('future.no_silent_downcasting', True)

df = pd.read_csv("output_adam.csv")
df_label = pd.read_csv("mirai_labels.csv")
df["alert"] = df_label
df.head()

,frame.number,frame.len,frame.time,frame.time_epoch,frame.protocols,eth.src,eth.dst,eth.type,ip.src,ip.dst,...,imap.request.command,imap.response,ftp.request.command,ftp.request.arg,ftp.response.code,ftp.response.arg,ipv6.src,ipv6.dst,ipv6.plen,alert
0,1,60,"Oct 25, 2018 07:46:22.933899000 CEST",1.540446e+09,eth:ethertype:ip:tcp,5c:cf:7f:05:f2:c6,4c:09:d4:c6:12:7b,0x0800,192.168.2.108,52.24.43.67,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,2,60,"Oct 25, 2018 07:46:22.933904000 CEST",1.540446e+09,eth:ethertype:ip:tcp,5c:cf:7f:05:f2:c6,4c:09:d4:c6:12:7b,0x0800,192.168.2.108,52.25.66.250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,3,86,"Oct 25, 2018 07:46:22.934426000 CEST",1.540446e+09,eth:ethertype:ip:icmp:ip:tcp,4c:09:d4:c6:12:7b,5c:cf:7f:05:f2:c6,0x0800,192.168.2.1,192.168.2.108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,4,86,"Oct 25, 2018 07:46:22.934636000 CEST",1.540446e+09,eth:ethertype:ip:icmp:ip:tcp,4c:09:d4:c6:12:7b,5c:cf:7f:05:f2:c6,0x0800,192.168.2.1,192.168.2.108,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,5,60,"Oct 25, 2018 07:46:23.291054000 CEST",1.540446e+09,eth:ethertype:arp,48:02:2e:01:83:15,ff:ff:ff:ff:ff:ff,0x0806,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 764137 entries, 0 to 764136
Data columns (total 74 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   frame.number                764137 non-null  int64  
 1   frame.len                   764137 non-null  int64  
 2   frame.time                  764137 non-null  object 
 3   frame.time_epoch            764137 non-null  float64
 4   frame.protocols             764137 non-null  object 
 5   eth.src                     764137 non-null  object 
 6   eth.dst                     764137 non-null  object 
 7   eth.type                    760438 non-null  object 
 8   ip.src                      197701 non-null  object 
 9   ip.dst                      197701 non-null  object 
 10  ip.len                      197701 non-null  float64
 11  ip.ttl                      197701 non-null  float64
 12  ip.flags                    197701 non-null  object 
 13  ip.frag_offset

In [170]:
out_src = df.reset_index().groupby(['ip.src'])['index'].min()
out_dst = df.reset_index().groupby(['ip.dst'])['index'].min() + 0.5
print(out_dst)
out = pd.concat([out_src, out_dst], axis=0).sort_values()
out = out[~out.index.duplicated(keep='first')]
print(out)
out.iloc[:] = out.reset_index(name="ip").index
out = out.astype(int)
out.to_dict()
#df["ip.src"] = df["ip.src"].replace(mappings)
#df

ip.dst
104.199.156.58        23.5
104.209.134.106     9405.5
112.74.102.136       342.5
112.74.124.169      2700.5
114.114.114.114       14.5
                    ...   
61.220.62.219        425.5
65.212.71.102      12321.5
66.243.43.2         7533.5
74.125.31.99         132.5
8.8.8.8              296.5
Name: index, Length: 76, dtype: float64
192.168.2.108         0.0
52.24.43.67           0.5
52.25.66.250          1.5
192.168.2.1           2.0
192.168.2.101         5.0
                   ...   
224.0.0.252       41549.5
192.168.2.121     73459.5
192.168.2.122     74750.5
192.168.2.107    121658.0
192.168.4.1      193555.5
Name: index, Length: 78, dtype: float64


{'192.168.2.108': 0,
 '52.24.43.67': 1,
 '52.25.66.250': 2,
 '192.168.2.1': 3,
 '192.168.2.101': 4,
 '192.168.2.110': 5,
 '192.168.2.115': 6,
 '192.168.2.112': 7,
 '114.114.114.114': 8,
 '192.168.2.113': 9,
 '50.19.254.134': 10,
 '122.248.234.207': 11,
 '46.137.188.54': 12,
 '122.226.84.253': 13,
 '61.188.37.216': 14,
 '120.24.59.150': 15,
 '114.215.137.159': 16,
 '104.199.156.58': 17,
 '192.168.2.109': 18,
 '239.255.255.250': 19,
 '192.168.2.117': 20,
 '74.125.31.99': 21,
 '192.168.2.119': 22,
 '192.168.2.104': 23,
 '8.8.8.8': 24,
 '192.168.2.126': 25,
 '112.74.102.136': 26,
 '54.154.5.26': 27,
 '54.149.224.179': 28,
 '210.61.248.232': 29,
 '42.99.254.162': 30,
 '203.69.81.91': 31,
 '61.220.62.219': 32,
 '119.29.29.29': 33,
 '192.168.2.105': 34,
 '255.255.255.255': 35,
 '224.0.0.22': 36,
 '192.168.1.252': 37,
 '112.74.124.169': 38,
 '0.0.0.0': 39,
 '192.168.2.196': 40,
 '133.100.9.2': 41,
 '139.78.100.163': 42,
 '131.107.1.10': 43,
 '199.165.76.11': 44,
 '140.142.16.34': 45,
 '128.138

In [2]:
def ordinalize_packets(packet_data, column="ip"):
    src_col = f"{column}.src"
    dst_col = f"{column}.dst"
    if 'frame.number' not in packet_data.columns or src_col not in packet_data.columns or dst_col not in packet_data.columns:
        raise ValueError("DataFrame must contain 'frame.number' and  columns")

    out_src = packet_data.reset_index().groupby([src_col])['index'].min()
    out_dst = packet_data.reset_index().groupby([dst_col])['index'].min() + 0.5
    out = pd.concat([out_src, out_dst], axis=0).sort_values()
    out = out[~out.index.duplicated(keep='first')]
    out.iloc[:] = out.reset_index(name="ip").index
    out = out.astype(int)
    mappings = out.to_dict()
    return mappings
    #packet_data.loc[:, src_col] = packet_data[src_col].replace(mappings).astype(int)
    #packet_data.loc[:, dst_col] = packet_data[dst_col].replace(mappings).astype(int)

In [3]:
features = ["frame.number", "frame.len", 'frame.time', 'eth.src', 'eth.dst', 'ip.src', 'ip.dst', 'ip.len', 'ip.ttl',
            'tcp.srcport', 'tcp.dstport',
            'tcp.flags.ack', 'tcp.flags.fin', 'tcp.flags.reset', "alert", "udp.srcport", "udp.dstport"]
df = df[features]
df["frame.len"] /= 65535
df["ip.len"] /= 65535
df["ip.ttl"] /= 255
df["tcp.srcport"] /= 65535
df["tcp.dstport"] /= 65535
df["udp.srcport"] /= 65535
df["udp.dstport"] /= 65535
df["frame.time"] = df["frame.time"].str.replace(" CEST", "")
df["frame.time"] = pd.to_datetime(df["frame.time"])
df['dow_sin'] = np.sin(2 * np.pi * df["frame.time"].dt.dayofweek / 6)
df['dow_cos'] = np.cos(2 * np.pi * df["frame.time"].dt.dayofweek / 6)
df['hour_sin'] = np.sin(2 * np.pi * df["frame.time"].dt.hour / 24.0)
df['hour_cos'] = np.cos(2 * np.pi * df["frame.time"].dt.hour / 24.0)
df["minute_sin"] = np.sin(2 * np.pi * df["frame.time"].dt.minute / 60)
df["minute_cos"] = np.cos(2 * np.pi * df["frame.time"].dt.minute / 60)
df["second_sin"] = np.sin(2 * np.pi * df["frame.time"].dt.second / 60)
df["second_cos"] = np.cos(2 * np.pi * df["frame.time"].dt.second / 60)
df["ms_sin"] = np.sin(2 * np.pi * df["frame.time"].dt.microsecond / 1000000)
df["ms_cos"] = np.cos(2 * np.pi * df["frame.time"].dt.microsecond / 1000000)
df = df.fillna(0)  # only tcp is left
df = df.drop(columns=["frame.time"])

In [6]:
df.head()

,frame.number,frame.len,eth.src,eth.dst,ip.src,ip.dst,ip.len,ip.ttl,tcp.srcport,tcp.dstport,...,dow_sin,dow_cos,hour_sin,hour_cos,minute_sin,minute_cos,second_sin,second_cos,ms_sin,ms_cos
0,1,0.000916,5c:cf:7f:05:f2:c6,4c:09:d4:c6:12:7b,192.168.2.108,52.24.43.67,0.000671,1.00000,0.321569,0.001221,...,1.224647e-16,-1.0,0.965926,-0.258819,-0.994522,0.104528,0.743145,-0.669131,-0.403487,0.914985
1,2,0.000916,5c:cf:7f:05:f2:c6,4c:09:d4:c6:12:7b,192.168.2.108,52.25.66.250,0.000671,1.00000,0.313298,0.126345,...,1.224647e-16,-1.0,0.965926,-0.258819,-0.994522,0.104528,0.743145,-0.669131,-0.403458,0.914998
2,3,0.001312,4c:09:d4:c6:12:7b,5c:cf:7f:05:f2:c6,192.168.2.1,192.168.2.108,0.001099,0.25098,0.321569,0.001221,...,1.224647e-16,-1.0,0.965926,-0.258819,-0.994522,0.104528,0.743145,-0.669131,-0.400455,0.916316
3,4,0.001312,4c:09:d4:c6:12:7b,5c:cf:7f:05:f2:c6,192.168.2.1,192.168.2.108,0.001099,0.25098,0.313298,0.126345,...,1.224647e-16,-1.0,0.965926,-0.258819,-0.994522,0.104528,0.743145,-0.669131,-0.399246,0.916844
4,5,0.000916,48:02:2e:01:83:15,ff:ff:ff:ff:ff:ff,0,0,0.000000,0.00000,0.000000,0.000000,...,1.224647e-16,-1.0,0.965926,-0.258819,-0.994522,0.104528,0.669131,-0.743145,0.966915,-0.255099


In [4]:
from torch.utils.data import Dataset
from tqdm import trange
from multiprocessing import Pool


class NetworkDataset(Dataset):
    def __init__(self, data, window=128):
        p = Pool(processes=24)
        self.data = data
        self.window = window
        self.mappings_ip = p.map(self.ordinal_ip, trange(len(self)))
        self.mappings_eth = p.map(self.ordinal_eth, trange(len(self)))
        p.close()
        p.join()
            
    def ordinal_ip(self, i):
        return ordinalize_packets(self.data[i:i+self.window], "ip")
    def ordinal_eth(self, i):
        return ordinalize_packets(self.data[i:i+self.window], "eth")
    
    def __getitem__(self, index):
        packets = self.data[index:index + self.window]
        labels = packets['alert'].to_numpy().astype(np.float32)
        packets.loc[:, "eth.src"] = packets["eth.src"].replace(self.mappings_eth[index]).astype(int)
        packets.loc[:, "eth.dst"] = packets["eth.dst"].replace(self.mappings_eth[index]).astype(int)
        packets.loc[:, "ip.src"] = packets["ip.src"].replace(self.mappings_ip[index]).astype(int)
        packets.loc[:, "ip.dst"] = packets["ip.dst"].replace(self.mappings_ip[index]).astype(int)
        packets = packets.drop(columns=['alert', "frame.number"]).fillna(0)
        return (packets[["eth.src", "eth.dst"]].astype(int).values, packets[["ip.src", "ip.dst"]].astype(int).values,
                packets.drop(columns=["ip.src", "ip.dst", "eth.src", "eth.dst"]).values.astype("float32"), labels)

    def __len__(self):
        return self.data.shape[0] - self.window

In [24]:
nds = NetworkDataset(df, 1024)
nds[12][1]

  8%|▊         | 63600/763113 [00:02<00:24, 28136.59it/s]Process ForkPoolWorker-181:
Process ForkPoolWorker-177:
Process ForkPoolWorker-178:
Process ForkPoolWorker-183:
Process ForkPoolWorker-179:
Process ForkPoolWorker-192:
Process ForkPoolWorker-186:
Process ForkPoolWorker-173:
Process ForkPoolWorker-191:
Process ForkPoolWorker-190:
Process ForkPoolWorker-170:
Process ForkPoolWorker-187:
Process ForkPoolWorker-188:
Process ForkPoolWorker-169:
Process ForkPoolWorker-171:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-184:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-189:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Process ForkPoolWorker-185:
Process ForkPoolWorker-180:


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/pool.py", line 125, in worker
    result = (True, func(*args, **kwds))
                    ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/frame.py", line 6417, in reset_index
    new_obj = self.copy(deep=None)
              ^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/pool.py", line 48, in mapstar
    return list(map(*args))
           ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/internals/managers.py", line 363, in apply
    applied = getattr(b, f)(**kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pandas/core/internals/blocks.py", line 796, in copy
    values = values.copy()
             ^^^^^^^^^^^^^
  File "/tmp/ipykernel_16290/2123840817.py", line 16, in ordinal_ip
    return ordinalize_packets(self.data[i:i+self.window], "ip")
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [5]:
from torch.nn.utils.parametrizations import weight_norm


class ResBlock(nn.Module):
    def __init__(self, channels, kernel_size, dilation):
        super(ResBlock, self).__init__()
        self.conv1 = weight_norm(
            nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=kernel_size, dilation=dilation))
        self.conv2 = weight_norm(
            nn.Conv1d(in_channels=channels, out_channels=channels, kernel_size=kernel_size, dilation=dilation))
        self.pad = nn.ZeroPad1d((kernel_size - 1) // 2 * dilation)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        t = self.pad(x)
        t = F.relu(self.conv1(t))
        t = self.pad(t)
        t = F.relu(self.conv2(t))
        return self.dropout(t) + x

In [6]:
class TCN(nn.Module):
    def __init__(self, channels, kernel_size, blocks):
        super(TCN, self).__init__()
        self.blocks = nn.Sequential(*[ResBlock(channels, kernel_size, 2**i) for i in range(blocks)])
        
    def forward(self, x):
        return self.blocks(x)

In [16]:
rb =TCN(5, 5, 2)
rb(torch.randn(2, 5, 50))

tensor([[[ 0.4930,  2.8734,  0.7040,  1.4007,  0.5570,  0.8242,  0.5057,
           0.6803,  0.7938,  0.3710, -0.5096,  0.6751, -0.0863, -0.2357,
           0.1171,  1.3098, -0.0148,  1.3838, -0.1150, -1.2456, -0.6741,
           0.1877, -0.4860,  1.2179,  2.0754, -0.3600, -0.2070, -0.7151,
           0.6399, -0.0777,  0.3021, -0.3402,  0.4434,  1.5218,  0.5087,
           0.1899, -0.6611, -0.4585,  2.4599,  0.4934,  0.4422,  0.5624,
           2.2803, -1.5272,  0.7496, -0.0379,  1.2823,  0.8802, -0.7981,
           0.6152],
         [ 1.8333, -0.5736, -1.3496,  1.0371,  1.0337,  0.0684, -0.2696,
           2.1811,  0.2497,  1.6957, -2.0771,  0.3720,  2.0790, -0.3636,
           2.0944, -0.1222,  2.0948,  0.6292,  1.3397,  0.6612, -0.6909,
           1.0906, -1.5022,  0.5549, -0.8636, -1.5973, -0.5547,  1.2952,
           1.8265,  0.9957,  2.0427, -0.8961,  1.0709, -0.0629,  0.7529,
          -2.8566,  0.5028, -0.9615, -0.4447, -1.2141,  0.3986,  0.6234,
          -1.0600, -0.6818,  0.

In [7]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.eth_embed = nn.Embedding(128, 32)
        self.ip_embed = nn.Embedding(128, 32)
        self.total_embed = nn.Linear(148, 64)
        self.conv = TCN(128, 5, 3)
        self.projection = nn.Linear(64, 1)

    def forward(self, eth, ip, packet):
        B = packet.size(0)
        L = packet.size(1)
        eth = self.eth_embed(eth).view(B, L, -1)
        ip = self.ip_embed(ip).view(B, L, -1)
        pk_embed = self.total_embed(torch.cat((eth, ip, packet), dim=-1))
        processed = nn.functional.relu(self.conv(pk_embed))
        preds = self.projection(processed).squeeze(-1)
        return nn.functional.sigmoid(preds)

In [12]:
dataset = NetworkDataset(df, 128) # preparation of dataset

100%|██████████| 764009/764009 [00:58<00:00, 12965.52it/s]


In [37]:
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm
from accelerate import Accelerator

accelerator = Accelerator(mixed_precision="bf16")
model = Model()
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, drop_last=True, num_workers=24, pin_memory=True,
                        persistent_workers=True)
optimizer = Adam(model.parameters(), lr=1e-4)
w = torch.tensor([5.28], dtype=torch.float32, device="cuda")
loss_fn = nn.BCELoss(weight=w)

model, dataloader, optimizer = accelerator.prepare(model, dataloader, optimizer)

for epoch in range(20):
    count = 0
    total = 0

    for eth, ip, pkt, labels in tqdm(dataloader):
        optimizer.zero_grad()
        pkt = model(eth, ip, pkt)
        loss = loss_fn(pkt, labels)
        count += 1
        total += loss.item()
        loss.backward()
        optimizer.step()

    print("Epoch {} Loss {}".format(epoch, total / count))

100%|██████████| 11937/11937 [01:32<00:00, 128.50it/s]


Epoch 0 Loss 0.05115495374315928


100%|██████████| 11937/11937 [01:32<00:00, 128.44it/s]


Epoch 1 Loss 0.007229339445790845


100%|██████████| 11937/11937 [01:34<00:00, 126.88it/s]


Epoch 2 Loss 0.005456030471377554


100%|██████████| 11937/11937 [01:34<00:00, 126.12it/s]


Epoch 3 Loss 0.003010150454189608


 44%|████▍     | 5250/11937 [00:41<00:53, 125.68it/s]Exception in thread Thread-19 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.11/threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/pin_memory.py", line 59, in _pin_memory_loop
    do_one_step()
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/pin_memory.py", line 35, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/multiprocessing/reductions.py", line 541, in rebuild_storage_fd
    fd = df.detach()
         ^^^^^^^^^^^
 

KeyboardInterrupt: 

In [39]:
ip, eth, pkt, labels = dataset[121500]
ip = torch.from_numpy(ip).cuda().unsqueeze(0)
eth = torch.from_numpy(eth).cuda().unsqueeze(0)
pkt = torch.from_numpy(pkt).cuda().unsqueeze(0)
labels = torch.from_numpy(labels).cuda().unsqueeze(0)

pred = model(ip, eth, pkt)
print(pred > 0.5)
labels

tensor([[False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
          True,  True, False, False, False,  True, False, False, False, False,
          True,  True,  True,  True, False, False, F

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
         1., 1.]], device='cuda:0')

In [68]:
loss_fn(pred.reshape(-1, 1), labels.reshape(-1, 1))

tensor(0.7049, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)

In [32]:
c = df["alert"].value_counts()
print(c)
c[1]/c[0]

alert
1.0    642516
0.0    121621
Name: count, dtype: int64


5.282936335007935